In [1]:
!pip install opencv-python mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 47.6 MB/s eta 0:00:00


In [3]:
import cv2
import mediapipe as mp
import numpy as np

# 初始化 MediaPipe Pose 模組
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 開啟影片檔案
video_path = '/content/w12_pushup.mp4'
cap = cv2.VideoCapture(video_path)

# 檢查影片是否成功打開
if not cap.isOpened():
    print(f"無法開啟影片：{video_path}")
else:
    # 取得影片的寬度、高度和每秒幀數
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 定義影片寫入器
    out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # 計數器和狀態變數
    pushup_count = 0
    position = None  # 'down' 或 'up'

    while True:
        ret, frame = cap.read()
        if not ret:
            print("影片讀取完畢或發生錯誤，結束迴圈。")
            break

        # 將影像從 BGR 轉換為 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # 進行姿態估計
        results = pose.process(image)

        # 將影像轉回 BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            # 繪製姿態骨架
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # 取得所需的關鍵點座標
            landmarks = results.pose_landmarks.landmark
            shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

            # 計算肩膀、手肘和手腕之間的角度
            shoulder_elbow = np.array([shoulder.x, shoulder.y])
            elbow_wrist = np.array([elbow.x, elbow.y])
            shoulder_wrist = np.array([wrist.x, wrist.y])

            v1 = shoulder_elbow - elbow_wrist
            v2 = shoulder_wrist - elbow_wrist

            angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
            angle = np.degrees(angle)

            # 根據角度判斷伏地挺身的上下動作
            if angle > 160:
                if position == 'down':
                    pushup_count += 1
                    position = 'up'
            elif angle < 90:
                position = 'down'

            # 在影像上顯示計數
            cv2.putText(image, f'Push-ups: {pushup_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # 寫入當前幀到影片
        out.write(image)

    cap.release()
    out.release()


影片讀取完畢或發生錯誤，結束迴圈。


In [ ]:
fps

25.0